# CNN Classifier

### Import Libraries

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Resizing, Conv2D, MaxPooling2D, Dense, Flatten, Rescaling, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from skimage import exposure
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

#### Model training done with CPU. If CUDA and cuDNN are set up, a GPU can also be used to speed up model training. 

In [ ]:
print(f'TensorFlow Version: {tf.__version__}')
print(tf.config.experimental.list_physical_devices())

### Global Variables

In [ ]:
data_directory: str = 'data'
batch_size: int = 32
image_height: int = 256
image_width: int = 256
random_state: int = 111

# True balances dataset to 10k of each class, false uses 20k generated images and 10k real images
balance_dataset_TF: bool = True

### Creating Training and Test Sets

In [ ]:
# Helper Functions
def load_dataset(dataset_dir):
    '''
    Loads and preprocesses image dataset. 
    Adjust preprocessing steps here and the preprocess_image function.
    Requires only the image path. 

    Returns two arrays of the images and the labels. 
    '''
    images = []
    labels = []
    class_names = sorted(os.listdir(dataset_dir))
    
    for ii, class_name in enumerate(class_names):
        class_dir = os.path.join(dataset_dir, class_name)
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            image = cv2.imread(image_path)  # Load image
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Make grayscale
            image = cv2.resize(image, (image_height, image_width))  # Resize
            #image = exposure.equalize_hist(image) # Histogram equalization
            images.append(image)
            labels.append(ii)  # Assign a label to the image based on the class index
    
    return np.array(images), np.array(labels)

def preprocess_image(image_path):
    '''
    Loads and preprocesses a single image for classification.
    Requires only the image path.
    '''
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # Make grayscale
    image = cv2.resize(image, (image_height, image_width)) # Resize
    image = exposure.equalize_hist(image) # Histogram equalization
    return image 
    
def balance_dataset(balance_dataset_TF, images, labels):
    ''' 
    Optional: Randomly under sample majority class (AI-generated) for more balaanced dataset
    Uses a random state from the global variable random_state.
    Change balance_dataset_TF to True for balanced or False for imbalanced. 

    Returns either a balanced 50/50 dataset or the original 2/3 fake, 1/3 real dataset as arrays. 
    '''
    if balance_dataset_TF == True:
        indicies_of_majority_class = [x for x, label in enumerate(labels) if label == 0]
        random.seed(random_state) # Set seed for reproducibility
        random_indicies = random.sample(indicies_of_majority_class, 10000) # Randomly select 10000 fake images
        random_undersampled_images = [images[index] for index in random_indicies]
        random_undersampled_labels = [labels[index] for index in random_indicies]
        
        #Add back real images to randomly selected generated images
        balanced_image_data = random_undersampled_images + [images[index] for index, label in enumerate(labels) if label == 1]
        balanced_label_data = random_undersampled_labels + [label for label in labels if label == 1]
        return np.array(balanced_image_data), np.array(balanced_label_data)
    
    elif balance_dataset_TF == False:
        return np.array(images), np.array(labels)

In [ ]:
images, labels = load_dataset(data_directory)
images, labels = balance_dataset(balance_dataset_TF, images, labels)
print(f"Images shape: {images.shape}")
print(f"Labels shape: {labels.shape}")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size = 0.3, random_state = random_state)

### Create CNN Model Using the Keras Sequential API

In [ ]:
# Custom architecture for baseline testing
# Images can be any size

model = Sequential([
  Rescaling(1./255, input_shape=(image_height, image_width, 1)), # 1 at the end for Grayscale, 3 for RGB
  Conv2D(16, 3, activation = 'relu'),
  MaxPooling2D(),
  Conv2D(32, 3, activation = 'relu'),
  MaxPooling2D(),
  Conv2D(32, 3, activation = 'relu'),
  MaxPooling2D(),
  Dropout(0.25),
  Flatten(),
  Dense(16, activation = 'relu'),
  Dense(1, activation = 'softmax')
])

model.compile(optimizer = 'adam', 
              loss = tf.losses.BinaryCrossentropy(), 
              metrics = ['accuracy'])

model.summary()

In [ ]:
# Mimicking the LiNet CNN Architecture
# Expects 233x233 images

LiNet = Sequential([
    # Layer 1
    Conv2D(32, (7,7), padding = 'same', activation = 'relu', input_shape = (image_height, image_width, 1)),
    
    # Layer 2
    Conv2D(64, (7,7), strides = 2, padding = 'same', activation = 'relu'),
    BatchNormalization(),
    MaxPooling2D(3, strides = 2, padding = 'same'),

    # Layer 3
    Conv2D(48, (5, 5), padding = 'same', activation = 'relu'),
    BatchNormalization(),
    MaxPooling2D((3, 3), strides = 2, padding = 'same'),

    # Layer 4
    Conv2D(64, (3, 3), padding = 'same', activation = 'relu'),
    BatchNormalization(),
    MaxPooling2D((3, 3), strides = 2, padding = 'same'),

    # Fully Connected Layers
    Flatten(),
    Dense(4096, activation = 'relu'),
    Dropout(0.5),
    Dense(4096, activation = 'relu'),
    Dropout(0.5),
    Dense(1, activation = 'softmax' )
])

LiNet.compile(optimizer = 'adam', 
              loss = tf.losses.BinaryCrossentropy(), 
              metrics = ['accuracy'])

LiNet.summary()

In [ ]:
# Mimicking the AlexNet CNN Architecture
# Expects 227x227 images

AlexNet = Sequential([
    Rescaling(1./255, input_shape = (image_height, image_width, 1)),

    # Layer 1
    Conv2D(96, (11, 11), strides = (4, 4), activation = 'relu', input_shape = (image_height, image_width, 1)), # 1 for Grayscale
    MaxPooling2D((3, 3), strides = (2, 2)),
    BatchNormalization(),

    # Layer 2
    Conv2D(256, (5, 5), padding = 'same', activation = 'relu'),
    MaxPooling2D((3, 3), strides = (2, 2)),
    BatchNormalization(),

    # Layer 3
    Conv2D(384, (3, 3), padding = 'same', activation = 'relu'),
    Conv2D(384, (3, 3), padding = 'same', activation = 'relu'),
    Conv2D(256, (3, 3), padding = 'same', activation = 'relu'),
    MaxPooling2D((3, 3), strides = (2, 2)),
    BatchNormalization(),
    Flatten(),

    # Fully connected layers
    Dense(4096, activation = 'relu'),
    Dropout(0.5),
    Dense(4096, activation = 'relu'),
    Dropout(0.5),
    Dense(1, activation = 'softmax')
])

AlexNet.compile(optimizer = 'adam',
              loss = tf.losses.BinaryCrossentropy(),
              metrics = ['accuracy'])

AlexNet.summary()

In [ ]:
# Using a pretrained model from TensorFlow called ResNet50
# Expects 224x224 images

from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D

resnet_base = ResNet50(input_shape = (image_height, image_width, 3),
                       include_top = False,
                       weights = 'imagenet',
                       classes = 2) # 1 for Grayscale

# Prevent the pretrained layers from being overwritten
resnet_base.trainable = False

# Establish the ResNet model with custom Dense layer
resnet_model = Sequential([
    resnet_base,
    GlobalAveragePooling2D(),
    Dense(1, activation = 'softmax') # Only need to train this layer
    ])

resnet_model.compile(optimizer = 'adam',
              loss = tf.losses.BinaryCrossentropy(),
              metrics = ['accuracy'])

# Images need to be in a specific format for the ResNet model
resnet_input_images = preprocess_input(images)

X_train, X_test, y_train, y_test = train_test_split(resnet_input_images, labels, test_size = 0.3, random_state = random_state)

resnet_model.summary()

In [ ]:
# This cell starts the training process and kicks off a TensorBoard instance for performance visualizations.

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='logs')
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), callbacks=[tensorboard_callback])

In [ ]:
fig = plt.figure()
plt.plot(history.history['loss'], color='teal', label='loss')
plt.plot(history.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(history.history['accuracy'], color='teal', label='accuracy')
plt.plot(history.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

None of the model architectures really learned how to distinguish between the two classes.
 
The loss vs. validation and accuracy vs. validation accuracy were flat, showing no changes for any of the architectures. 

The models are not saved off because they are no better than randomly guessing classes

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

pre = Precision()
re = Recall()
acc = BinaryAccuracy()

for batch in X_test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)


print(pre.result(), re.result(), acc.result())

In [ ]:
img = cv2.imread('test-fake5.jpg')
plt.imshow(img)
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
yhat = model.predict(np.expand_dims(resize/255, 0))
if yhat > 0.5: 
    print(f'Predicted class is Real: {yhat}')
else:
    print(f'Predicted class is Fake: {yhat}')